In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sys
import itertools
from itertools import islice
from itertools import product
import pyspark
from pyspark.sql import functions as F
from pyspark.sql import Window, DataFrame
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import udf
import pyspark.sql.functions as F
from pyspark.sql.functions import col, row_number, expr,when, lit
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import FloatType, IntegerType, LongType, StringType,StructType, StructField
from pyspark.sql import Row
from pyspark.ml.feature import StringIndexer
from google.cloud import storage
from pyspark.storagelevel import StorageLevel
from pyspark.ml.feature import StringIndexerModel
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover

import os
import time


# Tạo session mới
print("Creating new Spark session...")
spark = (
    SparkSession.builder
    .appName("ALS")
    .getOrCreate()
)

print("Spark master:", spark.sparkContext.master)


Creating new Spark session...
Spark master: yarn


25/05/09 17:05:51 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
# spark.stop()
conf = dict(spark.sparkContext.getConf().getAll())

print("spark.executor.instances:", conf.get("spark.executor.instances"))
print("spark.executor.cores:", conf.get("spark.executor.cores"))
print("spark.executor.memory:", conf.get("spark.executor.memory"))
print("spark.driver.memory:", conf.get("spark.driver.memory"))
print("spark.sql.shuffle.partitions:", conf.get("spark.sql.shuffle.partitions"))
print("spark.kryoserializer.buffer.max:", conf.get("spark.kryoserializer.buffer.max"))
print("spark.kryoserializer.buffer:", conf.get("spark.kryoserializer.buffer"))


spark.executor.instances: 2
spark.executor.cores: 4
spark.executor.memory: 25g
spark.driver.memory: 25g
spark.sql.shuffle.partitions: 100
spark.kryoserializer.buffer.max: 1024m
spark.kryoserializer.buffer: 64m


In [6]:
bucket_name = "team-15"

tfidf = spark.read.parquet(f"gs://{bucket_name}/tfidf/")


In [7]:
df_reviews = spark.read.parquet(f"gs://{bucket_name}/dataset/")
df_reviews.cache()
df_reviews.show(10)

25/05/09 17:06:46 WARN DAGScheduler: Broadcasting large task binary with size 295.7 MiB


+------+----------+----------+
|rating|user_index|item_index|
+------+----------+----------+
|   5.0| 4067184.0| 5114046.0|
|   5.0| 4067184.0| 5066546.0|
|   5.0| 4067184.0| 2067665.0|
|   5.0| 4067184.0| 5026158.0|
|   5.0| 4067184.0| 5120623.0|
|   4.0| 5834754.0| 2379309.0|
|   4.0| 5834754.0| 3588803.0|
|   5.0| 5834754.0| 1034071.0|
|   5.0| 5834754.0| 5137683.0|
|   5.0| 5834754.0| 4445808.0|
+------+----------+----------+
only showing top 10 rows



In [10]:
from pyspark.sql.functions import col, concat_ws
df_item = df_item.withColumn("combined_text", concat_ws(" ", col("title"), col("main_category")))



In [11]:
# df_item.show(5)

In [12]:

# Tokenization
tokenizer = Tokenizer(inputCol = "combined_text", outputCol = "tokens")
content_df = tokenizer.transform(df_item)


In [13]:
# Removing Stop Words

stopwords_remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens")
content_df = stopwords_remover.transform(content_df)

In [14]:
# Convert Text to Numerical Features
hashingTF = HashingTF(inputCol="filtered_tokens", outputCol="rawFeatures", numFeatures=65536)
featurizedData = hashingTF.transform(content_df)


In [15]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
tfidf = idfModel.transform(featurizedData)

In [16]:
tfidf.cache()
tfidf.show(1)

25/05/09 17:07:12 WARN DAGScheduler: Broadcasting large task binary with size 63.7 MiB


+-------------+--------------------+--------------+-------------+----------+--------------------+
|main_category|               title|average_rating|rating_number|item_index|            features|
+-------------+--------------------+--------------+-------------+----------+--------------------+
|  Amazon Home|DriftAway Juliann...|           4.7|          447| 2590986.0|(65536,[12524,156...|
+-------------+--------------------+--------------+-------------+----------+--------------------+
only showing top 1 row



In [17]:

tfidf_features = tfidf.select("item_index", "features")
# # 1. Lọc tương tác tích cực (ví dụ: rating >= 4)
# positive_interactions = df_reviews.filter(col("rating") >= 4.0)
# # 2. Join với đặc trưng sản phẩm
# user_item_features = positive_interactions.join(tfidf_features, "item_index")


In [18]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import col, udf
from pyspark.sql import functions as F
from pyspark.ml.feature import Normalizer
from pyspark.sql.types import DoubleType
import math


In [19]:
EXPECTED_VECTOR_SIZE = 65536 

def average_sparse_vectors(vectors):
    if not vectors:
        return Vectors.sparse(EXPECTED_VECTOR_SIZE, {})
    
    size = EXPECTED_VECTOR_SIZE 
    


    dict_sum = {}
    num_vectors = 0 
    for v_sparse in vectors:
        if v_sparse is not None and hasattr(v_sparse, 'indices') and hasattr(v_sparse, 'values'):
            num_vectors +=1
            for i, val in zip(v_sparse.indices, v_sparse.values):
                dict_sum[i] = dict_sum.get(i, 0.0) + val
        else:
            
            pass
            
    if num_vectors == 0 or not dict_sum:
         return Vectors.sparse(size, {})

    for i in dict_sum:
        dict_sum[i] = dict_sum[i] / num_vectors
            
    sorted_indices_values = sorted(dict_sum.items())
    
    final_indices = [item[0] for item in sorted_indices_values]
    final_values = [item[1] for item in sorted_indices_values]
    
    return Vectors.sparse(size, final_indices, final_values)

average_vectors_udf = udf(average_sparse_vectors, VectorUDT())

print("Creating user profiles...")
user_profiles_df = user_item_features.groupBy("user_index") \
    .agg(average_vectors_udf(F.collect_list("features")).alias("user_profile_vector"))



user_profiles_df.show(5, truncate=False)

In [20]:
user_profiles_df = spark.read.parquet(f"gs://{bucket_name}/user_profiles/")
user_profiles_df.cache()
user_profiles_df.show(5)

25/05/09 17:07:42 WARN DAGScheduler: Broadcasting large task binary with size 232.0 MiB


+----------+--------------------+
|user_index| user_profile_vector|
+----------+--------------------+
|     118.0|(65536,[1509,1598...|
|     299.0|(65536,[770,1851,...|
|     305.0|(65536,[393,1118,...|
|     323.0|(65536,[32,74,106...|
|     493.0|(65536,[178,343,4...|
+----------+--------------------+
only showing top 5 rows



In [21]:
# user_profiles_df.write.mode("overwrite").parquet(f"gs://{bucket_name}/user_profiles/")


In [32]:
# Danh sách số
user_id = [
    2715975.0,
    541078.0,
    1717487.0,
    4142059.0,
    377882.0,
    5525177.0,
    4429053.0,
    1753193.0,
    4157055.0,
    1726869.0
]

# Tạo DataFrame
user_id  = spark.createDataFrame([(n,) for n in user_id], ["user_index"])

# Hiển thị
user_id.show()

+----------+
|user_index|
+----------+
| 1726869.0|
+----------+



In [33]:
def recommend_for_user(user_profile_vector, all_items_normalized_df, k=10):
    """
    Gợi ý top-k sản phẩm cho một user dựa trên user_profile_vector.
    all_items_normalized_df phải có cột "norm_features".
    """

    broadcast_user_profile = spark.sparkContext.broadcast(user_profile_vector.toArray())

    def cosine_similarity_with_user_profile(item_norm_features):
        profile_arr = broadcast_user_profile.value
        item_arr = item_norm_features.toArray()

        dot = float(np.dot(item_arr, profile_arr))
        norm_item = float(np.linalg.norm(item_arr))
        norm_profile = float(np.linalg.norm(profile_arr))

        if norm_item > 0 and norm_profile > 0:
            return float(dot / (norm_item * norm_profile))
        return 0.0

    cosine_sim_udf_user = udf(cosine_similarity_with_user_profile, DoubleType())

    recommendations_df = all_items_normalized_df.withColumn(
        "similarity_to_user", cosine_sim_udf_user(col("norm_features"))
    )

    top_k_recs = recommendations_df.orderBy(col("similarity_to_user").desc()).limit(k)
    return top_k_recs.select("item_index", "title", "main_category", "similarity_to_user")

In [34]:

# # --- Sử dụng ---
# # Lấy hồ sơ của một người dùng cụ thể để thử nghiệm
# target_user_id = 4067184.0 # ID của người dùng bạn muốn gợi ý
# single_user_profile_row = user_profiles_df.filter(col("user_index") == target_user_id).first()

# Join để lấy vector hồ sơ cho từng user
selected_user_profiles = user_profiles_df.join(user_id, on="user_index", how="inner")

# Chuẩn hóa vector item một lần (nếu chưa làm)
normalizer = Normalizer(inputCol="features", outputCol="norm_features", p=2.0) # L2 norm
all_items_normalized_df = normalizer.transform(tfidf.select("item_index", "title", "main_category", "features"))
all_items_normalized_df.cache()

# List để chứa DataFrame kết quả gợi ý cho từng người dùng
all_user_recommendations_dfs = []

# --- Vòng lặp qua từng người dùng trong danh sách đã chọn ---
print("Bắt đầu tạo gợi ý cho từng người dùng...")

for user_row in selected_user_profiles.collect():
    current_user_id = user_row["user_index"]
    current_user_vector = user_row["user_profile_vector"]

    print(f"Đang tạo gợi ý cho user_index: {int(current_user_id)}") # Cast to int for cleaner print

    # 1. Lấy danh sách các item mà người dùng HIỆN TẠI đã tương tác
    interactions_for_current_user = df_reviews.filter(col("user_index") == current_user_id).select("item_index").distinct()

    # 2. Lọc bỏ các item đã tương tác khỏi danh sách toàn bộ item đã chuẩn hóa
    items_to_recommend_from = all_items_normalized_df.join(
        interactions_for_current_user,
        "item_index",
        "left_anti" # Giữ lại những item trong all_items_normalized_df KHÔNG có trong interactions_for_current_user
    )

    # 3. Thực hiện recommendation chỉ trên tập các item chưa tương tác cho người dùng HIỆN TẠI
    # Giả định recommend_for_user nhận user_vector và DataFrame các item đã được chuẩn hóa (đã lọc)
    user_recs_df = recommend_for_user(current_user_vector, items_to_recommend_from, k=10)

    # Thêm cột user_index để biết gợi ý này thuộc về ai
    if user_recs_df.isEmpty():
         print(f"Không tìm thấy gợi ý (sau khi lọc) cho user_index: {int(current_user_id)}")
    else:
        user_recs_df = user_recs_df.withColumn("user_index", lit(current_user_id))
        all_user_recommendations_dfs.append(user_recs_df)
        print(f"Đã tạo xong gợi ý cho user_index: {int(current_user_id)}")




Bắt đầu tạo gợi ý cho từng người dùng...


25/05/10 01:49:14 WARN DAGScheduler: Broadcasting large task binary with size 232.0 MiB
25/05/10 02:02:08 WARN DAGScheduler: Broadcasting large task binary with size 232.1 MiB


Đang tạo gợi ý cho user_index: 1726869


25/05/10 02:04:03 WARN DAGScheduler: Broadcasting large task binary with size 63.7 MiB
25/05/10 02:04:13 WARN DAGScheduler: Broadcasting large task binary with size 295.8 MiB
25/05/10 02:13:23 WARN DAGScheduler: Broadcasting large task binary with size 295.8 MiB
25/05/10 02:14:05 WARN DAGScheduler: Broadcasting large task binary with size 359.5 MiB
25/05/10 02:16:28 WARN DAGScheduler: Broadcasting large task binary with size 359.5 MiB
25/05/10 02:16:30 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_8_11 !
25/05/10 02:16:30 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_8_36 !
25/05/10 02:16:30 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_248_4 !
25/05/10 02:16:30 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_28_118 !
25/05/10 02:16:30 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_154_114 !
25/05/10 02:16:30 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_28_2

Đã tạo xong gợi ý cho user_index: 1726869


In [35]:
# --- Kết hợp kết quả từ tất cả người dùng ---
if all_user_recommendations_dfs:
    # Union tất cả các DataFrame kết quả lại thành một DataFrame duy nhất
    final_recommendations_df = reduce(DataFrame.unionAll, all_user_recommendations_dfs)

    print("\n--- Kết quả gợi ý cuối cùng cho tất cả người dùng đã chọn ---")
    # Hiển thị kết quả
    # Sắp xếp theo user_index để dễ nhìn (tùy chọn)
    final_recommendations_df.orderBy("user_index").show(truncate=False)
else:
    print("\nKhông có gợi ý nào được tạo ra cho bất kỳ người dùng nào trong danh sách.")


--- Kết quả gợi ý cuối cùng cho tất cả người dùng đã chọn ---


25/05/10 02:16:40 WARN DAGScheduler: Broadcasting large task binary with size 63.7 MiB
25/05/10 02:16:51 WARN DAGScheduler: Broadcasting large task binary with size 295.8 MiB
25/05/10 02:25:37 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_154_147 !
25/05/10 02:25:37 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_248_87 !
25/05/10 02:25:37 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_17_183 !
25/05/10 02:25:37 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_28_104 !
25/05/10 02:25:37 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_8_261 !
25/05/10 02:25:37 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_154_129 !
25/05/10 02:25:37 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_8_221 !
25/05/10 02:25:37 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_8_327 !
25/05/10 02:25:37 WARN BlockManagerMasterEndpoint: No more replica

+----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------+-------------------+----------+
|item_index|title                                                                                                                                                                                                |main_category           |similarity_to_user |user_index|
+----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------+-------------------+----------+
|3317188.0 |LED Bulb                                                                                                                                                                                   

In [ ]:




final_recommendations_df.write.mode("append").parquet(f"gs://{bucket_name}/content-based_rec/")

25/05/10 02:42:11 WARN DAGScheduler: Broadcasting large task binary with size 63.7 MiB
25/05/10 02:42:21 WARN DAGScheduler: Broadcasting large task binary with size 295.8 MiB
25/05/10 03:02:57 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_17_86 !
25/05/10 03:02:57 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_17_44 !
25/05/10 03:02:57 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_248_114 !
25/05/10 03:02:57 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_8_11 !
25/05/10 03:02:57 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_248_97 !
25/05/10 03:02:57 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_8_36 !
25/05/10 03:02:57 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_8_327 !
25/05/10 03:02:57 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_248_4 !
25/05/10 03:02:57 WARN BlockManagerMasterEndpoint: No more replicas avai